In [1]:
import os 
import numpy
from scipy.io import loadmat

## Dataset
The dataset contains the EEG recordings of 15 subjects. For each subject, we have 15 different recordings, each one collected while watching a different movie clip. Each clip is associated to an emotional state amon {sad: -1, neutral: 0, happy: 1}. EEG recordings comprises 62 channels.

Data have been preprocessed by downsampling signals to 200Hz, segmentating the signals such that it corresponds to the length of the movie and applying a band-pass filter at 0-75Hz. Since recordings are about 4 minutes long and are now sampled at 200Hz, they contain roughly 48k time points each.

In [2]:
data = loadmat("data/Preprocessed_EEG/1_20131027.mat")

In [5]:
labels = loadmat("data/Preprocessed_EEG/label.mat")
labels["label"].shape

(1, 15)

In [13]:
print(data.keys())
data["djc_eeg3"].shape

dict_keys(['__header__', '__version__', '__globals__', 'djc_eeg1', 'djc_eeg2', 'djc_eeg3', 'djc_eeg4', 'djc_eeg5', 'djc_eeg6', 'djc_eeg7', 'djc_eeg8', 'djc_eeg9', 'djc_eeg10', 'djc_eeg11', 'djc_eeg12', 'djc_eeg13', 'djc_eeg14', 'djc_eeg15'])


(62, 41201)